# 1. Build a convolutional neural network to classify object proposals (N+1 classes)

In [10]:
import json
import os
from sklearn.model_selection import train_test_split
from PIL import Image

# Load splits.json
with open('../Potholes/splits.json', 'r') as f:
    splits = json.load(f)

# Extract train and test splits
train_files = splits['train']
test_files = splits['test']

# Further split the train set into train and validation
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42)

In [11]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Paths
from Dataset import PotholeDataset
from tqdm import tqdm
img_dir = '../Potholes/annotated-images'
annotations_dir = '../Potholes/labeled_proposals'

# Initialize datasets
train_dataset = PotholeDataset(img_dir, annotations_dir, train_files, transform=transform)
val_dataset = PotholeDataset(img_dir, annotations_dir, val_files, transform=transform)
test_dataset = PotholeDataset(img_dir, annotations_dir, test_files, transform=transform)

# WeightedRandomSampler for training
labels = [label for _, label in tqdm(train_dataset,desc = "sampler for split")]

sampler for split: 100%|██████████| 142273/142273 [09:18<00:00, 254.87it/s]


In [ ]:
import torch
from PotholeCNN import PotholeCNN
from TrainLoop import train


# Initialize model, optimizer, and load data
model = PotholeCNN()  # Your model class
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_loader = ...  # DataLoader for training
val_loader = ...    # DataLoader for validation
test_loader = ...   # DataLoader for testing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
train(model, optimizer, epochs=10, train_loader=train_loader, test_loader=test_loader, device=device)




Epoch 1/10


Training Epoch 1:   0%|          | 0/142273 [00:00<?, ?batch/s]


TypeError: tuple indices must be integers or slices, not str